# Notebook Objective:

Objective of the notebook is to look at the different pretrained embeddings provided in the dataset and to see how they are useful in the model building process.

First let us import the necessary modules and read the input data.

In [11]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [20]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
print("Train shape : ", train_df.shape)
print("Test shape : ", test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11816342070403021042
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3164969369
locality {
  bus_id: 1
  links {
  }
}
incarnation: 379451362428587691
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


# Next steps are as follows:

* Split the training dataset into train and val sample. Cross validation is a time consuming process and so let us do simple train val split.
* Fill up the missing values in the text column with 'na'
* Tokenize the text column and convert them to vector sequences
* Pad the sequence as needed - if the number of words in the text is greater than 'max_len' trunacate them to 'max_len' or if the number of words in the text is lesser than 'max_len' add zeros for remaining values.

In [22]:
# split to train and val
train_df, val_df = train_test_split(train_df, test_size = 0.1, random_state = 2018)

# some config values
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max numbers of words in a question to use

# fill up missing values

train_X = train_df['question_text'].fillna("_na_").values
val_X = val_df['question_text'].fillna("_na_").values
test_X = test_df['question_text'].fillna("_na_").values

# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

#Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [23]:
train_X.shape

(1175509, 100)

In [24]:
EMBEDDING_FILE = "embeddings/glove.840B.300d/glove.840B.300d.txt"
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE,encoding="utf8"))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total para

In [25]:
model.fit(train_X, train_y, batch_size=512, epochs=5, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/5


 103424/1175509 [=>............................] - ETA: 1:00:05 - loss: 1.3555 - acc: 0.06 - ETA: 32:33 - loss: 1.1541 - acc: 0.0537 - ETA: 23:21 - loss: 0.9963 - acc: 0.22 - ETA: 18:39 - loss: 0.8868 - acc: 0.38 - ETA: 15:52 - loss: 0.7961 - acc: 0.50 - ETA: 13:59 - loss: 0.7230 - acc: 0.57 - ETA: 12:39 - loss: 0.6663 - acc: 0.62 - ETA: 11:39 - loss: 0.6180 - acc: 0.66 - ETA: 10:52 - loss: 0.5835 - acc: 0.69 - ETA: 10:14 - loss: 0.5485 - acc: 0.71 - ETA: 9:43 - loss: 0.5182 - acc: 0.7370 - ETA: 9:17 - loss: 0.4949 - acc: 0.753 - ETA: 8:55 - loss: 0.4772 - acc: 0.767 - ETA: 8:37 - loss: 0.4630 - acc: 0.779 - ETA: 8:21 - loss: 0.4527 - acc: 0.788 - ETA: 8:07 - loss: 0.4391 - acc: 0.798 - ETA: 7:55 - loss: 0.4255 - acc: 0.807 - ETA: 7:43 - loss: 0.4159 - acc: 0.814 - ETA: 7:33 - loss: 0.4053 - acc: 0.821 - ETA: 7:24 - loss: 0.3968 - acc: 0.827 - ETA: 7:15 - loss: 0.3893 - acc: 0.832 - ETA: 7:07 - loss: 0.3826 - acc: 0.837 - ETA: 7:00 - loss: 0.3761 - acc: 0.841 - ETA: 6:53 - loss: 0.3733

 312320/1175509 [======>.......................] - ETA: 3:51 - loss: 0.1493 - acc: 0.942 - ETA: 3:51 - loss: 0.1492 - acc: 0.942 - ETA: 3:51 - loss: 0.1492 - acc: 0.942 - ETA: 3:50 - loss: 0.1491 - acc: 0.942 - ETA: 3:50 - loss: 0.1490 - acc: 0.942 - ETA: 3:50 - loss: 0.1489 - acc: 0.942 - ETA: 3:50 - loss: 0.1488 - acc: 0.942 - ETA: 3:50 - loss: 0.1487 - acc: 0.942 - ETA: 3:50 - loss: 0.1486 - acc: 0.942 - ETA: 3:50 - loss: 0.1486 - acc: 0.942 - ETA: 3:49 - loss: 0.1487 - acc: 0.942 - ETA: 3:49 - loss: 0.1486 - acc: 0.942 - ETA: 3:49 - loss: 0.1485 - acc: 0.942 - ETA: 3:49 - loss: 0.1485 - acc: 0.942 - ETA: 3:49 - loss: 0.1485 - acc: 0.942 - ETA: 3:49 - loss: 0.1486 - acc: 0.942 - ETA: 3:49 - loss: 0.1485 - acc: 0.942 - ETA: 3:49 - loss: 0.1485 - acc: 0.942 - ETA: 3:48 - loss: 0.1484 - acc: 0.942 - ETA: 3:48 - loss: 0.1483 - acc: 0.942 - ETA: 3:48 - loss: 0.1482 - acc: 0.942 - ETA: 3:48 - loss: 0.1482 - acc: 0.942 - ETA: 3:48 - loss: 0.1481 - acc: 0.942 - ETA: 3:48 - loss: 0.1480 - ac

 416768/1175509 [=========>....................] - ETA: 3:25 - loss: 0.1383 - acc: 0.946 - ETA: 3:25 - loss: 0.1383 - acc: 0.946 - ETA: 3:25 - loss: 0.1382 - acc: 0.946 - ETA: 3:25 - loss: 0.1382 - acc: 0.946 - ETA: 3:25 - loss: 0.1382 - acc: 0.946 - ETA: 3:25 - loss: 0.1381 - acc: 0.946 - ETA: 3:24 - loss: 0.1381 - acc: 0.946 - ETA: 3:24 - loss: 0.1381 - acc: 0.946 - ETA: 3:24 - loss: 0.1381 - acc: 0.946 - ETA: 3:24 - loss: 0.1382 - acc: 0.946 - ETA: 3:24 - loss: 0.1381 - acc: 0.946 - ETA: 3:24 - loss: 0.1381 - acc: 0.946 - ETA: 3:24 - loss: 0.1380 - acc: 0.946 - ETA: 3:24 - loss: 0.1380 - acc: 0.946 - ETA: 3:23 - loss: 0.1379 - acc: 0.946 - ETA: 3:23 - loss: 0.1379 - acc: 0.946 - ETA: 3:23 - loss: 0.1379 - acc: 0.946 - ETA: 3:23 - loss: 0.1378 - acc: 0.946 - ETA: 3:23 - loss: 0.1378 - acc: 0.946 - ETA: 3:23 - loss: 0.1377 - acc: 0.946 - ETA: 3:23 - loss: 0.1377 - acc: 0.946 - ETA: 3:23 - loss: 0.1376 - acc: 0.946 - ETA: 3:22 - loss: 0.1376 - acc: 0.946 - ETA: 3:22 - loss: 0.1375 - ac

 521216/1175509 [============>.................] - ETA: 3:00 - loss: 0.1319 - acc: 0.948 - ETA: 3:00 - loss: 0.1318 - acc: 0.948 - ETA: 3:00 - loss: 0.1318 - acc: 0.948 - ETA: 3:00 - loss: 0.1318 - acc: 0.948 - ETA: 3:00 - loss: 0.1318 - acc: 0.948 - ETA: 3:00 - loss: 0.1318 - acc: 0.948 - ETA: 3:00 - loss: 0.1318 - acc: 0.948 - ETA: 2:59 - loss: 0.1317 - acc: 0.948 - ETA: 2:59 - loss: 0.1317 - acc: 0.948 - ETA: 2:59 - loss: 0.1316 - acc: 0.948 - ETA: 2:59 - loss: 0.1316 - acc: 0.948 - ETA: 2:59 - loss: 0.1316 - acc: 0.948 - ETA: 2:59 - loss: 0.1315 - acc: 0.948 - ETA: 2:59 - loss: 0.1315 - acc: 0.948 - ETA: 2:59 - loss: 0.1314 - acc: 0.948 - ETA: 2:58 - loss: 0.1314 - acc: 0.948 - ETA: 2:58 - loss: 0.1313 - acc: 0.948 - ETA: 2:58 - loss: 0.1313 - acc: 0.948 - ETA: 2:58 - loss: 0.1312 - acc: 0.948 - ETA: 2:58 - loss: 0.1312 - acc: 0.948 - ETA: 2:58 - loss: 0.1312 - acc: 0.948 - ETA: 2:58 - loss: 0.1312 - acc: 0.948 - ETA: 2:58 - loss: 0.1311 - acc: 0.948 - ETA: 2:58 - loss: 0.1311 - ac

...

 835584/1175509 [====================>.........] - ETA: 1:51 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:50 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:49 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - acc: 0.980 - ETA: 1:48 - loss: 0.0499 - ac

 940032/1175509 [======================>.......] - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:24 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:23 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:22 - loss: 0.0502 - acc: 0.980 - ETA: 1:21 - loss: 0.0502 - ac

1050112/1175509 [=========================>....] - ETA: 58s - loss: 0.0507 - acc: 0.97 - ETA: 58s - loss: 0.0507 - acc: 0.97 - ETA: 58s - loss: 0.0507 - acc: 0.97 - ETA: 58s - loss: 0.0507 - acc: 0.97 - ETA: 58s - loss: 0.0508 - acc: 0.97 - ETA: 58s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 57s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 56s - loss: 0.0508 - acc: 0.97 - ETA: 55s - loss: 0.0508 - acc: 0.97 - ETA: 55s - loss: 0.0508 - acc: 0.97 - ETA: 55s - loss: 0.0508 - acc: 0.97 -

1160192/1175509 [============================>.] - ETA: 31s - loss: 0.0511 - acc: 0.97 - ETA: 31s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 30s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 29s - loss: 0.0511 - acc: 0.97 - ETA: 28s - loss: 0.0511 - acc: 0.97 - ETA: 28s - loss: 0.0511 - acc: 0.97 - ETA: 28s - loss: 0.0512 - acc: 0.97 - ETA: 28s - loss: 0.0512 - acc: 0.97 - ETA: 28s - loss: 0.0512 - acc: 0.97 - ETA: 28s - loss: 0.0512 - acc: 0.97 - ETA: 28s - loss: 0.0512 - acc: 0.97 - ETA: 28s - loss: 0.0512 - acc: 0.97 -

1175509/1175509 [==============================] - ETA: 3s - loss: 0.0514 - acc: 0.979 - ETA: 3s - loss: 0.0514 - acc: 0.979 - ETA: 3s - loss: 0.0514 - acc: 0.979 - ETA: 3s - loss: 0.0514 - acc: 0.979 - ETA: 3s - loss: 0.0514 - acc: 0.979 - ETA: 3s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 2s - loss: 0.0514 - acc: 0.979 - ETA: 1s - loss: 0.0514 - acc: 0.979 - ETA: 1s - loss: 0.0514 - acc: 0.979 - ETA: 1s - loss: 0.0514 - acc: 0.979 - ETA: 1s - loss: 0.0515 - acc: 0.979 - ETA: 1s - loss: 0.0515 - acc: 0.979 - ETA: 1s - loss: 0.0515 - acc: 0.979 - ETA: 1s - loss: 0.0515 - acc: 0.979 - ETA: 1s - loss: 0.0515 - acc: 0.979 - ETA: 0s - loss: 0.0515 - acc: 0.979 - ETA: 0s - loss: 0.0515 - acc: 0.979 - ETA: 0s - loss: 0.0515 - acc: 0.979 -

In [26]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)

130613/130613 [==============================] - ETA: 1: - ETA: 43s - ETA: 31 - ETA: 25 - ETA: 21 - ETA: 19 - ETA: 17 - ETA: 16 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - E

In [27]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.6365679264555669
F1 score at threshold 0.11 is 0.63900372054568
F1 score at threshold 0.12 is 0.6404511984960051
F1 score at threshold 0.13 is 0.6422811363516577
F1 score at threshold 0.14 is 0.6423823451209786
F1 score at threshold 0.15 is 0.6436472988937816
F1 score at threshold 0.16 is 0.6442276422764228
F1 score at threshold 0.17 is 0.645189472534673
F1 score at threshold 0.18 is 0.6460844202300369
F1 score at threshold 0.19 is 0.6468174761587935
F1 score at threshold 0.2 is 0.6463482831897999
F1 score at threshold 0.21 is 0.6470819967358883
F1 score at threshold 0.22 is 0.646905463613693
F1 score at threshold 0.23 is 0.6470319634703197
F1 score at threshold 0.24 is 0.6469473321463443
F1 score at threshold 0.25 is 0.646854764107308
F1 score at threshold 0.26 is 0.6477199930114729
F1 score at threshold 0.27 is 0.6473033905252679
F1 score at threshold 0.28 is 0.6462753418198962
F1 score at threshold 0.29 is 0.6453450424256809
F1 score at threshold 0.3 i

Results seem to be better than the model without pretrained embeddings.

In [75]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA: 20 - ETA: 11 - ETA: 8 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 65us/step


In [76]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

# Wiki News FastText Embeddings:

Now let us use the FastText embeddings trained on Wiki News corpus in place of Glove embeddings and rebuild the model.

In [77]:
EMBEDDING_FILE = 'embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE,encoding='utf8') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total para

In [78]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2


 103424/1175509 [=>............................] - ETA: 52:24 - loss: 0.7877 - acc: 0.05 - ETA: 28:34 - loss: 0.7346 - acc: 0.35 - ETA: 20:33 - loss: 0.6874 - acc: 0.54 - ETA: 16:31 - loss: 0.6462 - acc: 0.64 - ETA: 14:06 - loss: 0.6092 - acc: 0.70 - ETA: 12:28 - loss: 0.5768 - acc: 0.74 - ETA: 11:19 - loss: 0.5452 - acc: 0.76 - ETA: 10:28 - loss: 0.5214 - acc: 0.78 - ETA: 9:48 - loss: 0.4965 - acc: 0.8047 - ETA: 9:15 - loss: 0.4735 - acc: 0.817 - ETA: 8:49 - loss: 0.4551 - acc: 0.827 - ETA: 8:27 - loss: 0.4382 - acc: 0.837 - ETA: 8:08 - loss: 0.4206 - acc: 0.845 - ETA: 7:52 - loss: 0.4070 - acc: 0.852 - ETA: 7:38 - loss: 0.3988 - acc: 0.856 - ETA: 7:26 - loss: 0.3925 - acc: 0.860 - ETA: 7:15 - loss: 0.3824 - acc: 0.865 - ETA: 7:05 - loss: 0.3719 - acc: 0.870 - ETA: 6:57 - loss: 0.3645 - acc: 0.874 - ETA: 6:49 - loss: 0.3558 - acc: 0.878 - ETA: 6:42 - loss: 0.3503 - acc: 0.881 - ETA: 6:36 - loss: 0.3435 - acc: 0.884 - ETA: 6:30 - loss: 0.3405 - acc: 0.886 - ETA: 6:24 - loss: 0.3337 - a

 521216/1175509 [============>.................] - ETA: 2:59 - loss: 0.1303 - acc: 0.950 - ETA: 2:58 - loss: 0.1302 - acc: 0.950 - ETA: 2:58 - loss: 0.1302 - acc: 0.950 - ETA: 2:58 - loss: 0.1302 - acc: 0.950 - ETA: 2:58 - loss: 0.1302 - acc: 0.950 - ETA: 2:58 - loss: 0.1301 - acc: 0.950 - ETA: 2:58 - loss: 0.1301 - acc: 0.950 - ETA: 2:58 - loss: 0.1301 - acc: 0.950 - ETA: 2:58 - loss: 0.1301 - acc: 0.950 - ETA: 2:57 - loss: 0.1301 - acc: 0.950 - ETA: 2:57 - loss: 0.1300 - acc: 0.950 - ETA: 2:57 - loss: 0.1300 - acc: 0.950 - ETA: 2:57 - loss: 0.1299 - acc: 0.950 - ETA: 2:57 - loss: 0.1299 - acc: 0.950 - ETA: 2:57 - loss: 0.1299 - acc: 0.950 - ETA: 2:57 - loss: 0.1298 - acc: 0.950 - ETA: 2:57 - loss: 0.1298 - acc: 0.950 - ETA: 2:57 - loss: 0.1298 - acc: 0.950 - ETA: 2:56 - loss: 0.1298 - acc: 0.950 - ETA: 2:56 - loss: 0.1298 - acc: 0.950 - ETA: 2:56 - loss: 0.1297 - acc: 0.950 - ETA: 2:56 - loss: 0.1298 - acc: 0.950 - ETA: 2:56 - loss: 0.1298 - acc: 0.950 - ETA: 2:56 - loss: 0.1297 - ac

 625664/1175509 [==============>...............] - ETA: 2:34 - loss: 0.1264 - acc: 0.951 - ETA: 2:34 - loss: 0.1264 - acc: 0.951 - ETA: 2:34 - loss: 0.1264 - acc: 0.951 - ETA: 2:34 - loss: 0.1263 - acc: 0.951 - ETA: 2:34 - loss: 0.1263 - acc: 0.951 - ETA: 2:34 - loss: 0.1263 - acc: 0.951 - ETA: 2:34 - loss: 0.1263 - acc: 0.951 - ETA: 2:33 - loss: 0.1263 - acc: 0.951 - ETA: 2:33 - loss: 0.1263 - acc: 0.951 - ETA: 2:33 - loss: 0.1263 - acc: 0.951 - ETA: 2:33 - loss: 0.1262 - acc: 0.951 - ETA: 2:33 - loss: 0.1262 - acc: 0.951 - ETA: 2:33 - loss: 0.1262 - acc: 0.951 - ETA: 2:33 - loss: 0.1262 - acc: 0.951 - ETA: 2:33 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1261 - acc: 0.951 - ETA: 2:32 - loss: 0.1260 - ac

 730112/1175509 [=================>............] - ETA: 2:10 - loss: 0.1238 - acc: 0.952 - ETA: 2:10 - loss: 0.1238 - acc: 0.952 - ETA: 2:09 - loss: 0.1238 - acc: 0.952 - ETA: 2:09 - loss: 0.1238 - acc: 0.952 - ETA: 2:09 - loss: 0.1238 - acc: 0.952 - ETA: 2:09 - loss: 0.1238 - acc: 0.952 - ETA: 2:09 - loss: 0.1238 - acc: 0.952 - ETA: 2:09 - loss: 0.1237 - acc: 0.952 - ETA: 2:09 - loss: 0.1237 - acc: 0.952 - ETA: 2:09 - loss: 0.1237 - acc: 0.952 - ETA: 2:09 - loss: 0.1237 - acc: 0.952 - ETA: 2:08 - loss: 0.1237 - acc: 0.952 - ETA: 2:08 - loss: 0.1237 - acc: 0.952 - ETA: 2:08 - loss: 0.1237 - acc: 0.952 - ETA: 2:08 - loss: 0.1236 - acc: 0.952 - ETA: 2:08 - loss: 0.1236 - acc: 0.952 - ETA: 2:08 - loss: 0.1236 - acc: 0.952 - ETA: 2:08 - loss: 0.1237 - acc: 0.952 - ETA: 2:08 - loss: 0.1237 - acc: 0.952 - ETA: 2:07 - loss: 0.1236 - acc: 0.952 - ETA: 2:07 - loss: 0.1236 - acc: 0.952 - ETA: 2:07 - loss: 0.1236 - acc: 0.952 - ETA: 2:07 - loss: 0.1236 - acc: 0.952 - ETA: 2:07 - loss: 0.1236 - ac

...

 835584/1175509 [====================>.........] - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:47 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:46 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:45 - loss: 0.0921 - acc: 0.963 - ETA: 1:44 - loss: 0.0921 - ac

 940544/1175509 [=======================>......] - ETA: 1:22 - loss: 0.0921 - acc: 0.963 - ETA: 1:22 - loss: 0.0921 - acc: 0.963 - ETA: 1:22 - loss: 0.0921 - acc: 0.963 - ETA: 1:22 - loss: 0.0921 - acc: 0.963 - ETA: 1:22 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:21 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:20 - loss: 0.0921 - acc: 0.963 - ETA: 1:19 - loss: 0.0921 - acc: 0.963 - ETA: 1:19 - loss: 0.0921 - acc: 0.963 - ETA: 1:19 - loss: 0.0921 - ac

1050624/1175509 [=========================>....] - ETA: 57s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 56s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 55s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 - ETA: 54s - loss: 0.0922 - acc: 0.96 -

1160704/1175509 [============================>.] - ETA: 30s - loss: 0.0923 - acc: 0.96 - ETA: 30s - loss: 0.0923 - acc: 0.96 - ETA: 30s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 29s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 28s - loss: 0.0923 - acc: 0.96 - ETA: 27s - loss: 0.0923 - acc: 0.96 - ETA: 27s - loss: 0.0923 - acc: 0.96 - ETA: 27s - loss: 0.0923 - acc: 0.96 - ETA: 27s - loss: 0.0923 - acc: 0.96 - ETA: 27s - loss: 0.0923 - acc: 0.96 - ETA: 27s - loss: 0.0923 - acc: 0.96 -

1175509/1175509 [==============================] - ETA: 3s - loss: 0.0925 - acc: 0.962 - ETA: 3s - loss: 0.0925 - acc: 0.962 - ETA: 3s - loss: 0.0925 - acc: 0.962 - ETA: 3s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 2s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 1s - loss: 0.0925 - acc: 0.962 - ETA: 0s - loss: 0.0925 - acc: 0.962 - ETA: 0s - loss: 0.0925 - acc: 0.962 - ETA: 0s - loss: 0.0925 - acc: 0.962 - ETA: 0s - loss: 0.0925 - acc: 0.962 - ETA: 0s - loss: 0.0925 - acc: 0.962 -

In [79]:
pred_fasttext_val_y = model.predict([val_X], batch_size=1024, verbose=1)

130613/130613 [==============================] - ETA: 49 - ETA: 28 - ETA: 21 - ETA: 18 - ETA: 16 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 

In [80]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_fasttext_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.5969949357552422
F1 score at threshold 0.11 is 0.6041033109178909
F1 score at threshold 0.12 is 0.6118893363684118
F1 score at threshold 0.13 is 0.6186009112838381
F1 score at threshold 0.14 is 0.624556616643929
F1 score at threshold 0.15 is 0.6291149175862228
F1 score at threshold 0.16 is 0.6355701548568747
F1 score at threshold 0.17 is 0.6393762183235868
F1 score at threshold 0.18 is 0.6439646428053905
F1 score at threshold 0.19 is 0.6481880509304603
F1 score at threshold 0.2 is 0.6518327207708353
F1 score at threshold 0.21 is 0.6547684882610226
F1 score at threshold 0.22 is 0.6578035859820701
F1 score at threshold 0.23 is 0.6598864223025297
F1 score at threshold 0.24 is 0.6631127835267064
F1 score at threshold 0.25 is 0.664765525982256
F1 score at threshold 0.26 is 0.6670941541092231
F1 score at threshold 0.27 is 0.6690095155709342
F1 score at threshold 0.28 is 0.6707843673134737
F1 score at threshold 0.29 is 0.6711920529801324
F1 score at threshold 0.

In [81]:
pred_fasttext_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 66us/step


In [83]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

# Paragram Embeddings:

In this section, we can use the paragram embeddings and build the model and make predictions.

In [84]:
EMBEDDING_FILE = 'embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [85]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2


 103424/1175509 [=>............................] - ETA: 1:00:00 - loss: 0.4700 - acc: 0.90 - ETA: 32:29 - loss: 0.4108 - acc: 0.9219 - ETA: 23:14 - loss: 0.3660 - acc: 0.92 - ETA: 18:32 - loss: 0.3306 - acc: 0.93 - ETA: 15:42 - loss: 0.3016 - acc: 0.93 - ETA: 13:49 - loss: 0.2971 - acc: 0.93 - ETA: 12:28 - loss: 0.2891 - acc: 0.93 - ETA: 11:28 - loss: 0.2787 - acc: 0.93 - ETA: 10:41 - loss: 0.2758 - acc: 0.93 - ETA: 10:03 - loss: 0.2717 - acc: 0.93 - ETA: 9:33 - loss: 0.2663 - acc: 0.9407 - ETA: 9:08 - loss: 0.2667 - acc: 0.940 - ETA: 8:46 - loss: 0.2654 - acc: 0.940 - ETA: 8:27 - loss: 0.2641 - acc: 0.940 - ETA: 8:11 - loss: 0.2614 - acc: 0.940 - ETA: 7:57 - loss: 0.2575 - acc: 0.940 - ETA: 7:44 - loss: 0.2531 - acc: 0.941 - ETA: 7:34 - loss: 0.2537 - acc: 0.940 - ETA: 7:24 - loss: 0.2502 - acc: 0.941 - ETA: 7:14 - loss: 0.2489 - acc: 0.940 - ETA: 7:06 - loss: 0.2484 - acc: 0.940 - ETA: 6:59 - loss: 0.2472 - acc: 0.940 - ETA: 6:52 - loss: 0.2470 - acc: 0.940 - ETA: 6:46 - loss: 0.2432

 416768/1175509 [=========>....................] - ETA: 3:23 - loss: 0.1305 - acc: 0.951 - ETA: 3:23 - loss: 0.1305 - acc: 0.951 - ETA: 3:22 - loss: 0.1305 - acc: 0.951 - ETA: 3:22 - loss: 0.1304 - acc: 0.951 - ETA: 3:22 - loss: 0.1304 - acc: 0.951 - ETA: 3:22 - loss: 0.1304 - acc: 0.951 - ETA: 3:22 - loss: 0.1304 - acc: 0.951 - ETA: 3:22 - loss: 0.1303 - acc: 0.951 - ETA: 3:22 - loss: 0.1303 - acc: 0.951 - ETA: 3:22 - loss: 0.1303 - acc: 0.951 - ETA: 3:21 - loss: 0.1302 - acc: 0.951 - ETA: 3:21 - loss: 0.1302 - acc: 0.951 - ETA: 3:21 - loss: 0.1301 - acc: 0.951 - ETA: 3:21 - loss: 0.1301 - acc: 0.951 - ETA: 3:21 - loss: 0.1301 - acc: 0.951 - ETA: 3:21 - loss: 0.1300 - acc: 0.951 - ETA: 3:21 - loss: 0.1300 - acc: 0.951 - ETA: 3:21 - loss: 0.1300 - acc: 0.951 - ETA: 3:21 - loss: 0.1300 - acc: 0.951 - ETA: 3:20 - loss: 0.1300 - acc: 0.951 - ETA: 3:20 - loss: 0.1300 - acc: 0.951 - ETA: 3:20 - loss: 0.1299 - acc: 0.951 - ETA: 3:20 - loss: 0.1299 - acc: 0.951 - ETA: 3:20 - loss: 0.1298 - ac

 521216/1175509 [============>.................] - ETA: 2:58 - loss: 0.1259 - acc: 0.952 - ETA: 2:58 - loss: 0.1260 - acc: 0.952 - ETA: 2:58 - loss: 0.1259 - acc: 0.952 - ETA: 2:58 - loss: 0.1259 - acc: 0.952 - ETA: 2:58 - loss: 0.1259 - acc: 0.952 - ETA: 2:58 - loss: 0.1259 - acc: 0.952 - ETA: 2:58 - loss: 0.1258 - acc: 0.952 - ETA: 2:57 - loss: 0.1258 - acc: 0.952 - ETA: 2:57 - loss: 0.1258 - acc: 0.952 - ETA: 2:57 - loss: 0.1258 - acc: 0.952 - ETA: 2:57 - loss: 0.1257 - acc: 0.952 - ETA: 2:57 - loss: 0.1257 - acc: 0.952 - ETA: 2:57 - loss: 0.1257 - acc: 0.952 - ETA: 2:57 - loss: 0.1257 - acc: 0.952 - ETA: 2:57 - loss: 0.1257 - acc: 0.952 - ETA: 2:56 - loss: 0.1257 - acc: 0.952 - ETA: 2:56 - loss: 0.1256 - acc: 0.952 - ETA: 2:56 - loss: 0.1256 - acc: 0.952 - ETA: 2:56 - loss: 0.1256 - acc: 0.952 - ETA: 2:56 - loss: 0.1255 - acc: 0.952 - ETA: 2:56 - loss: 0.1255 - acc: 0.952 - ETA: 2:56 - loss: 0.1255 - acc: 0.952 - ETA: 2:56 - loss: 0.1255 - acc: 0.952 - ETA: 2:55 - loss: 0.1255 - ac

 625664/1175509 [==============>...............] - ETA: 2:34 - loss: 0.1227 - acc: 0.953 - ETA: 2:34 - loss: 0.1227 - acc: 0.953 - ETA: 2:34 - loss: 0.1226 - acc: 0.953 - ETA: 2:34 - loss: 0.1226 - acc: 0.953 - ETA: 2:33 - loss: 0.1226 - acc: 0.953 - ETA: 2:33 - loss: 0.1226 - acc: 0.953 - ETA: 2:33 - loss: 0.1225 - acc: 0.953 - ETA: 2:33 - loss: 0.1225 - acc: 0.953 - ETA: 2:33 - loss: 0.1225 - acc: 0.953 - ETA: 2:33 - loss: 0.1225 - acc: 0.953 - ETA: 2:33 - loss: 0.1225 - acc: 0.953 - ETA: 2:33 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:32 - loss: 0.1225 - acc: 0.953 - ETA: 2:31 - loss: 0.1225 - acc: 0.953 - ETA: 2:31 - loss: 0.1224 - acc: 0.953 - ETA: 2:31 - loss: 0.1224 - ac

...

 835584/1175509 [====================>.........] - ETA: 1:49 - loss: 0.0942 - acc: 0.962 - ETA: 1:49 - loss: 0.0942 - acc: 0.962 - ETA: 1:48 - loss: 0.0942 - acc: 0.962 - ETA: 1:48 - loss: 0.0942 - acc: 0.962 - ETA: 1:48 - loss: 0.0941 - acc: 0.962 - ETA: 1:48 - loss: 0.0941 - acc: 0.962 - ETA: 1:48 - loss: 0.0942 - acc: 0.962 - ETA: 1:48 - loss: 0.0942 - acc: 0.962 - ETA: 1:48 - loss: 0.0941 - acc: 0.962 - ETA: 1:48 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:47 - loss: 0.0941 - acc: 0.962 - ETA: 1:46 - loss: 0.0941 - acc: 0.962 - ETA: 1:46 - loss: 0.0941 - acc: 0.962 - ETA: 1:46 - loss: 0.0941 - acc: 0.962 - ETA: 1:46 - loss: 0.0942 - acc: 0.962 - ETA: 1:46 - loss: 0.0941 - acc: 0.962 - ETA: 1:46 - loss: 0.0941 - ac

 940544/1175509 [=======================>......] - ETA: 1:23 - loss: 0.0941 - acc: 0.962 - ETA: 1:23 - loss: 0.0942 - acc: 0.962 - ETA: 1:23 - loss: 0.0942 - acc: 0.962 - ETA: 1:23 - loss: 0.0942 - acc: 0.962 - ETA: 1:23 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0941 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:22 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:21 - loss: 0.0942 - acc: 0.962 - ETA: 1:20 - loss: 0.0942 - acc: 0.962 - ETA: 1:20 - loss: 0.0941 - acc: 0.962 - ETA: 1:20 - loss: 0.0941 - ac

1050624/1175509 [=========================>....] - ETA: 57s - loss: 0.0942 - acc: 0.96 - ETA: 57s - loss: 0.0942 - acc: 0.96 - ETA: 57s - loss: 0.0942 - acc: 0.96 - ETA: 57s - loss: 0.0942 - acc: 0.96 - ETA: 57s - loss: 0.0942 - acc: 0.96 - ETA: 57s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 56s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 55s - loss: 0.0942 - acc: 0.96 - ETA: 54s - loss: 0.0942 - acc: 0.96 - ETA: 54s - loss: 0.0942 - acc: 0.96 - ETA: 54s - loss: 0.0942 - acc: 0.96 -

1160704/1175509 [============================>.] - ETA: 30s - loss: 0.0943 - acc: 0.96 - ETA: 30s - loss: 0.0942 - acc: 0.96 - ETA: 30s - loss: 0.0942 - acc: 0.96 - ETA: 30s - loss: 0.0942 - acc: 0.96 - ETA: 30s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 29s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 28s - loss: 0.0942 - acc: 0.96 - ETA: 27s - loss: 0.0942 - acc: 0.96 - ETA: 27s - loss: 0.0942 - acc: 0.96 - ETA: 27s - loss: 0.0942 - acc: 0.96 - ETA: 27s - loss: 0.0942 - acc: 0.96 -

1175509/1175509 [==============================] - ETA: 3s - loss: 0.0942 - acc: 0.962 - ETA: 3s - loss: 0.0942 - acc: 0.962 - ETA: 3s - loss: 0.0942 - acc: 0.962 - ETA: 3s - loss: 0.0942 - acc: 0.962 - ETA: 3s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 2s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 1s - loss: 0.0942 - acc: 0.962 - ETA: 0s - loss: 0.0942 - acc: 0.962 - ETA: 0s - loss: 0.0942 - acc: 0.962 - ETA: 0s - loss: 0.0942 - acc: 0.962 - ETA: 0s - loss: 0.0942 - acc: 0.962 - ETA: 0s - loss: 0.0942 - acc: 0.962 -

In [87]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)

130613/130613 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  -

In [88]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_paragram_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.6026412571046472
F1 score at threshold 0.11 is 0.6121715312847726
F1 score at threshold 0.12 is 0.6199965041076736
F1 score at threshold 0.13 is 0.6265317944833119
F1 score at threshold 0.14 is 0.6315360217489805
F1 score at threshold 0.15 is 0.6365060351976412
F1 score at threshold 0.16 is 0.641553579784745
F1 score at threshold 0.17 is 0.6463600076031173
F1 score at threshold 0.18 is 0.6498121930078012
F1 score at threshold 0.19 is 0.6534817853305988
F1 score at threshold 0.2 is 0.6576469422142928
F1 score at threshold 0.21 is 0.6608817775994382
F1 score at threshold 0.22 is 0.6632476896516706
F1 score at threshold 0.23 is 0.6647843942505134
F1 score at threshold 0.24 is 0.6658365758754864
F1 score at threshold 0.25 is 0.6684149795447393
F1 score at threshold 0.26 is 0.6707349429556911
F1 score at threshold 0.27 is 0.6719579557033303
F1 score at threshold 0.28 is 0.6739236393176279
F1 score at threshold 0.29 is 0.6747225107988409
F1 score at threshold 0

In [89]:
pred_paragram_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA: 20 - ETA: 11 - ETA: 8 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 65us/step


In [90]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

# Observations:

* Overall pretrained embeddings seem to give better results comapred to non-pretrained model.
* The performance of the different pretrained embeddings are almost similar.

# Final Blend:

* Though the results of the models with different pre-trained embeddings are similar, there is a good chance that they might capture different type of information from the data. So let us do a blend of these three models by averaging their predictions.

In [91]:
pred_val_y = 0.33*pred_glove_val_y + 0.33*pred_fasttext_val_y + 0.34*pred_paragram_val_y

In [92]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.6103994606438564
F1 score at threshold 0.11 is 0.6195530244197084
F1 score at threshold 0.12 is 0.6269484808454425
F1 score at threshold 0.13 is 0.6325169032373619
F1 score at threshold 0.14 is 0.6377959927140255
F1 score at threshold 0.15 is 0.6439720253809459
F1 score at threshold 0.16 is 0.6500518232356545
F1 score at threshold 0.17 is 0.6546305583416916
F1 score at threshold 0.18 is 0.6586924939467311
F1 score at threshold 0.19 is 0.6628358135651491
F1 score at threshold 0.2 is 0.6658041401273885
F1 score at threshold 0.21 is 0.6701649931883547
F1 score at threshold 0.22 is 0.6720179968301038
F1 score at threshold 0.23 is 0.6748371082842072
F1 score at threshold 0.24 is 0.6763446475195823
F1 score at threshold 0.25 is 0.6782324404134149
F1 score at threshold 0.26 is 0.6794243070362473
F1 score at threshold 0.27 is 0.6800839476941292
F1 score at threshold 0.28 is 0.6815539255637055
F1 score at threshold 0.29 is 0.6826110806363137
F1 score at threshold 

The result seems to better than individual pre-trained models and so we let us create a submission file using this model blend.

In [93]:
pred_test_y = 0.33*pred_glove_test_y + 0.33*pred_fasttext_test_y + 0.34*pred_paragram_test_y
pred_test_y = (pred_test_y>0.35).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)